In [1]:
DATA_FOLDER = './sub_data_2/'

import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
X = pickle.load(open(DATA_FOLDER + 'featureset.pickle', 'rb'))
Y = pickle.load(open(DATA_FOLDER + 'labels_int.pickle', 'rb'))

In [3]:
X_train_np, X_dev_test_np, y_train_np, y_dev_test_np = train_test_split(X, Y, test_size=0.3, random_state=42)

X_dev_np, X_test_np, y_dev_np, y_test_np = train_test_split(X_dev_test_np, y_dev_test_np, test_size=0.5, random_state=42)

In [4]:
# Normalization

normalizer = preprocessing.Normalizer().fit(X_train_np)

#X_train_np = normalizer.transform(X_train_np)
#X_dev_np = normalizer.transform(X_dev_np)
#X_test_np = normalizer.transform(X_test_np)

In [5]:
print(len(X_train_np), len(X_dev_np), len(X_test_np))

47773 10237 10238


In [6]:
X_train_np.shape

(47773, 2920)

In [7]:
_, feature_length = X_train_np.shape
class_length =  len(np.unique(Y))

print(feature_length, class_length)

2920 2699


In [8]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F

In [9]:
X_train = torch.from_numpy(X_train_np).float()
y_train = torch.LongTensor(y_train_np)

X_dev = torch.from_numpy(X_dev_np).float()
y_dev = torch.LongTensor(y_dev_np)

x, y = Variable(X_train), Variable(y_train)
x_dev, y_dev = Variable(X_dev), y_dev

In [12]:
net = torch.nn.Sequential(
    torch.nn.Linear(feature_length, 500),
    torch.nn.ReLU(),
    torch.nn.Linear(500, class_length),
    torch.nn.LogSoftmax()
)
print(net)

Sequential (
  (0): Linear (2920 -> 500)
  (1): ReLU ()
  (2): Linear (500 -> 2699)
  (3): LogSoftmax ()
)


In [13]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.02)
loss_func = torch.nn.NLLLoss()

In [14]:
for t in range(100):
    out = net(x)                 # input x and predict based on x
    loss = loss_func(out, y)     # must be (1. nn output, 2. target), the target label is NOT one-hotted

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    if t % 5 == 1 or t in [0, 2]:
        # plot and show learning process
        _, prediction = torch.max(out, 1)
        pred_y = prediction.data.numpy().squeeze()
        target_y = y.data.numpy()
        
        
        out_dev = net(x_dev)
        _, prediction_dev = torch.max(out_dev, 1)
        pred_y_dev = prediction_dev.data.numpy().squeeze()
        target_y_dev = y_dev.numpy()

        acc_train = sum(pred_y == target_y)/len(target_y)
        acc_dev = sum(pred_y_dev == target_y_dev)/len(target_y_dev)
        
        print('Epoch{0:3d} -> Loss={1:.6f}, Train Acc={2:.3f}, Dev Acc={3:.3f}'.format(t+1, loss.data[0], acc_train, acc_dev))


Epoch  1 -> Loss=7.906998, Train Acc=0.000, Dev Acc=0.273
Epoch  2 -> Loss=5.534091, Train Acc=0.280, Dev Acc=0.501
Epoch  3 -> Loss=3.791727, Train Acc=0.510, Dev Acc=0.574
Epoch  7 -> Loss=1.115723, Train Acc=0.829, Dev Acc=0.846
Epoch 12 -> Loss=0.312775, Train Acc=0.944, Dev Acc=0.916
Epoch 17 -> Loss=0.122690, Train Acc=0.975, Dev Acc=0.935
Epoch 22 -> Loss=0.062960, Train Acc=0.986, Dev Acc=0.942
Epoch 27 -> Loss=0.038710, Train Acc=0.991, Dev Acc=0.948


KeyboardInterrupt: 

In [165]:
from sklearn import preprocessing
import numpy as np
X_train = np.array([[ 51, -1.,  2000.],
                    [ 50.,  2.,  2500.],
                   [ 30.,  1., 200.]])

scaler = preprocessing.Normalizer().fit(X_train.T)
scaler.transform(X_train.T).T

array([[ 0.65835231, -0.40824829,  0.62347969],
       [ 0.64544344,  0.81649658,  0.77934961],
       [ 0.38726606,  0.40824829,  0.06234797]])

In [217]:
len(np.unique(Y))

2699

In [20]:
len(np.unique(np.array(Y)))

2699

In [22]:
y_train.shape

torch.Size([47773])

## Test with some example

In [78]:
y_test_np[0:5]

[394, 2685, 2579, 1584, 779]

In [76]:
doc_sender_dict = pickle.load(open(DATA_FOLDER + 'doc_sender_dict.pickle', 'rb'))
sender_id_all = pickle.load(open(DATA_FOLDER +'sender_id_all.pickle', 'rb'))

In [79]:
sender_id_all[2579]

'0727e23c-ae0f-45e8-a110-161e45c097b7'

In [80]:
for doc_id, sender_id in doc_sender_dict.items():
    if sender_id == '0727e23c-ae0f-45e8-a110-161e45c097b7':
        print(doc_id)

0093930f-35f6-4430-a6a8-420385a8245a
0140324f-5374-46fb-9687-39f27f0da163
01e32a17-40f7-46b1-a940-2a88a7e86865
023490aa-6e64-4fb9-ac6b-97447397c58c
02376ed0-5538-4aac-83e0-dbedae5c0166
02990909-4ea9-4723-ab06-74242e0f245f
03e792aa-b06c-49fc-919c-a4d552aa929f
045cf039-9b8f-4a21-bb64-9c583ff39385
068b004d-e5e4-48f1-83bd-4c4a6d083eb7
069d24ff-8b8a-4218-b34b-23db972a7b24
0704e170-1af0-4fd0-9139-8d61e00b8a43
07e17870-c96a-413b-afa3-63d4a80f2c74
092988b3-9050-4ed6-abea-d2500536f108
092be56a-3023-47ce-b103-b284fcd54b56
09c282ff-0e7c-49de-8ec7-d6b69736496d
09e51eec-b8d7-4a93-b295-662afdb21541
0a3cadb2-6bf9-4f5d-85ca-9bb7db8d2779
0a5d1ac9-6e76-4ccf-a2e7-ad76cc4a1875
0b589a3f-03ef-42d4-adbf-5211a9e8e42c
0b987093-9478-4acb-bf90-759ea972ba5b
0d20c19f-8c95-4e25-b48a-db85c4679767
0dd0d9ca-6322-4d93-9c0f-2254a1e7560e
0e3fe836-c4f4-43ce-8089-5b72dd28ebd7
10722895-c2ab-4289-a18f-73e3fc3224d4
10d0f56f-cd32-42c5-b932-c3afaa7c0ca1
10df3030-7c91-46fe-a4c0-01e4543f99ea
10f9a995-f801-4b72-a175-3e39a148c332
1

In [93]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

lexicon = pickle.load(open(DATA_FOLDER + 'lexicon.pickle', 'rb'))

def is_valid_word(word):
    import re
    if len(word) < 1:
        return False
    if re.search('[a-zA-Z]', word) == None:
        return False
    
    return True

def parse_single_file(j):    
    words = []
    for word_info in j.get('describedWords'):
        f = word_info.get('features')
        try:
            wordHight = word_info.get('bottom')
            pageHeight = float(f.get('pageHeight'))
            relativePosition = wordHight/pageHeight
            
            if relativePosition < 1.0/3 or relativePosition > 2.0/3:
                word = word_info.get('text')
                
                if is_valid_word(word):
                    local_words_list = tokenizer.tokenize(word.lower())
                    words += local_words_list
        except:
            pass
    return words

def get_sender_name_via_id(doc_id):
    import boto3
    import botocore
    import json
    
    b_name = 'ts-dev-cs-training-data'
    prefix = 'cache_v2_individual/ubl1/hocr13/ubllight1/hocrlight1/sendertrainingmatrices20170819/data/'
    key = prefix + doc_id + '.training'
    
    obj = boto3.resource('s3').Object(b_name, key)
    doc_str = obj.get()['Body'].read().decode('utf-8')
    doc_json = json.loads(doc_str)
    
    return doc_json['sender']

def get_feature_via_words(words):
    features = np.zeros(len(lexicon))
    for word in words:
        if word in lexicon:
            index_value = lexicon.index(word)
            features[index_value] += 1
    return np.array([features])

def get_words_via_id(doc_id):
    import boto3
    import botocore
    import json
    
    b_name = 'ts-dev-cs-training-data'
    prefix = 'rbp-research/data/e2e/2017-09-11/'
    key = prefix + doc_id + '.training.json'
    
    obj = boto3.resource('s3').Object(b_name, key)
    doc_str = obj.get()['Body'].read().decode('utf-8')
    doc_json = json.loads(doc_str)
    
    words = parse_single_file(doc_json)
    
    words_in_lexicon = []
    for word in words:
        if word in lexicon:
            words_in_lexicon.append(word)
    return words_in_lexicon


In [98]:
doc_id = '0093930f-35f6-4430-a6a8-420385a8245a'

print(get_sender_name_via_id(doc_id))

words = get_words_via_id(doc_id)
#words = ['paletten', 'paletten']
print(words)

f = get_feature_via_words(words)

f_tensor = torch.from_numpy(f).float()
f_variable = Variable(f_tensor)

out = net(f_variable)

_, prediction = torch.max(out, 1)
pred_y = prediction.data.numpy().squeeze()
print(pred_y)

Tara Paletten-Handel- und Reparaturdienst GmbH
['r', 'b', 'und', 'info', 'm', 'w', 'd', 'paletten', 't', 'g', 'p', 'u', 'd', 'w', 'w', 'h', 't', 'rechnung', 'zur', 'ta', 'r', 'm', 'd', 'e', 'e', 'e', 'r', 'd', 'w', 'd', 'em', 'd', 'bleibt', 'g', 'finance', 'w', 'w', 'g', 'unser', 'p', 'u', 'em', 'd', 'e', 'd', 'fax', 'rechnungsbetrag', 'ail', 'e', 'r', 'h', 'd', 'd', 'w', 'are', 'v', 'b', 'euro', 'b', 'w', 'ib', 'w', 'r', 'w', 'v', 'r', 'u', 't', 'h', 'n', 'r', 'u', 'bh', 'e', 'h', 'w', 'e', 'v', 'e', 'r', 'w', 'von', 'om', 'k', 'h', 'r', 'o', 'd', 'paletten', 'st', 'em', 'n', 'w', 'al', 'e', 'bis', 'tage', 'g', 'm', 'n', 'p', 'an', 'e', 'e', 'd', 'k', 'ie', 'g', 'ai', 'r', 'r', 'paletten', 'st', 'n', 'e', 'el', 'ib', 'd', 'fax', 'd', 'n', 'p', 'e', 'd', 'b', 'd', 'vollständigen', 'b', 'zahlbar', 'w', 'bh', 'w', 'tf', 'netto', 'c', 'u']
2579
